# Deep Learning for Image Super-resolution: A survey

*文章已经放在资源文件中*，文件位置：resources\literature\super resolution\sr-1.pdf

## Abstract

1. Image Super Resulotion（简称为 SR） 是一个在 CV 领域中提高图像和视频分辨率的技术。
2. SR 一般会分成三类：
- 监督的 SR
- 半监督的 SR
- 领域 SR
3. 这篇文章是为了介绍近几年来，Deep Learning 在 SR 领域的一些前沿的进展。并总结了 SR 上的性能评价指标和 benchmark 数据集。 
4. 讨论了一下未来的发展方向


## Introduction

SR 就是从低分辨率的图像中恢复出高分辨率的图像，是图像处理和 CV 中的一个很重要的领域。SR 的应用领域也很广泛：

- 医疗图像
- 监管和审查

SR 问题是 ill-posed。因为一张低分辨率的图像可以对应多张高分辨率的图像。

    如果一个问题是 well-posed，那么它需要满足：1. 解必须存在。2.解必须唯一。3.解能根据初始状态连续变化。否则这个问题就是 ill-posed。

SR 问题上一些传统典型的解决方法：

- prediction-based
- edge-based
- statistical-based
- patch-based
- sparse-representation

Deep Learning 的方法最近在 SR 问题上实现了 SOTA 的效果：

- CNN-based
- GAN

Deep Learning 中的方法差异主要体现在：

- 不同的网络结构
- 不同的损失函数
- 不同的学习策略

这篇文章主要的贡献点在于：

- 基于 Deep Learning 的视角，总结了 SR 问题上的技术、benchmark 数据集、性能指标
- 从层次和结构化的角度系统性的总结了 SR 问题上 Deep Learning 技术的进展
- 讨论了 SR 问题棉铃的挑战

## Problem Setting and Terminology

### Problem Definitions

SR 问题被定义成：

$$
    I_x = D(I_y; \delta)
$$

$D$ 表示的降低分辨率的函数， $I_y$ 表示高分辨率的图像， $\delta$ 表示函数 $D$ 的参数。通常，我们只有 $I_x$，我们就需要通过 $I_x$ ，恢复出真实的 $I_y$ ，也即使得 $\hat{I_y}$ 尽可能的和 $I_y$ 接近。所以：

$$
    I_y = F(I_x;\theta)
$$

其中 $F$ 是 SR 模型，$\theta$ 是 SR 模型的参数。尽管降低分辨率的过程有很多（人工压缩、传感器噪音、色斑噪音），但是大多数研究人员把 $D$ 看作是降采样：

$$
    D(I_y;\delta) = (I_y) \downarrow_s,{s} \subset \delta
$$

其中 $\downarrow_s$ 被看作是参数为 $s$ 的降采样操作。一般采用的降采样是 *带抗锯齿的双三次插值（bicubic interpolation with antialiasing）*。

还有人把 $D$ 看作是通过 *blur kernel* 处理后加入 *标准差为 $\sigma$ 高斯白噪声* 的结果：

$$
    D(I_y;\delta) = (I_y \otimes \kappa) \downarrow_s + n_{\sigma},\{\kappa, s, \sigma\} \subset \delta
$$

其中 $\kappa$ 是 blur kernel。$n_{\sigma}$ 是高斯白噪声。

SR 的目标函数是：

$$
    \hat{\theta} = arg \ min_{\theta} \ L(\hat{I_y}, I_y) + \lambda \phi(\theta)
$$

$\phi(\theta)$ 是正则项

## SR 的数据集

现在已经有很多的 SR 数据集，有些同时提供了 LR 和 HR 图像，而有些之提供了 HR 图像，这时候就需要通过 imresize 获取 LR 图像。图表 1 给出了 SR 采用的数据集：

![dataset-table](https://github.com/mayday-975/Machine-Learning-Literature-Review/blob/main/resources/images/super%20resolution/sr-1/sr-dataset-table.png?raw=true)

## 图像质量评价

图像质量评价（IQA）包含两部分，人的主观评价和客观的计算指标。很多时候客观的计算指标和人的主观评价并不时时一致，这就导致了主观评价和客观评价之间巨大的差异。

客观计算指标的三种类型：

1. full-reference performing assessment using reference images.(直接对比模型处理后的 SR 图像和真实的 SR 图像)
2. reduced-reference methods based on comparisons of extracted features.（对比模型处理的 SR 图像的特征和真实 SR 图像的特征）
3. no-reference methods without any reference images.（没有真实 SR 图像的评价指标）

### Peak Signal-to-Noise Ratio

PSNR 是最普遍有损失的转换重构质量评估方法之一（reconstruction quality measurement of lossy transformation）。其通过峰值像素值 $L$ 和两个图像之间的的 MSE 来定义。给定真实的图像 $I$ ， 其有 $N$ 个像素，以及重构的图像 $\hat{I}$ ，PSNR 的定义如下：

$$
    PSNR = 10 \cdot log_{10} (\frac{L^2}{\frac{1}{N} \sum_{i=1}^{N}(I(i)-\hat{I}(i)^2)})
$$

一般来说使用 8-bit 表示一个像素的时候，$L$ 的值最大为 255。PSNR 的通常只是表示像素之间的差异，但是没有评估视觉上的感知，所以在一些时候好的 PSNR 会导致人视觉上认为图像的效果是很差的。但是由于缺少精确的视觉感知评价指标， PSNR 依然在 SR 模型上广泛使用。

### Structural Similarity

人类的视觉系统是很容易的从图像中提取结构特征的，结构相似指数（structural similairty index）就被提出来评价两个图像之间的结构相似性。该评价指标基于：

- 亮度
- 对比度
- 结构

给一个图像 $I$ ，其具有 $N$ 个像素，亮度为 $\mu_{I} = \frac{1}{N} \sum_{i=1}^NI(i)$ ，对比度为 $\sigma_{I} = \sqrt{\frac{1}{N-1} \sum_{i=1}^N (I(i)-\mu_{I})^2}$，这两个指标分别被视为图像密度的均值和标准差。亮度和对比度的比较可以表示为：

$$
    C_{l}(I, \hat{I}) = \frac{2 \mu_I \mu_{\hat{I}} + C_1}{\mu_I^2 + \mu_{\hat{I}}^2 + C_1}, \qquad C_c(I, \hat{I}) = \frac{2 \sigma_I \sigma_{\hat{I}} + C_2}{\sigma_I^2 + \sigma_{\hat{I}}^2 + C_2}
$$

$C_1 = (k_1L)^2$，以及$C_2 =(k_2L)^2$ 是一个常量，并且有 $k_1, k_2 \ll 1$。

图像的结构相似性由经过标准化后的像素之间的相关性给出：

$$
\begin{align}
    \sigma_{I \ \hat{I}} = & \frac{1}{N-1} \sum_{i=1}^N (I(i)-\mu_{I})(\hat{I}(i) - \mu_{\hat{I}}) \nonumber \\
    C_s(I, \hat{I}) = & \frac{\sigma_{I \ \hat{I}} + C_3}{\sigma_I \sigma_{\hat{I}} + C_3} \nonumber
\end{align}
$$

最终的 SSIM 表示为：$SSIM(I, \hat{I}) = [C_l(I, \hat{I})]^{\alpha}[C_c(I, \hat{I}]^{\beta}[C_s(I, \hat{I})]^{\gamma}$，其中 $\alpha,\ \beta, \ \gamma$ 是超参数。

### Mean Opinion Score

该评价找一群人对图片进行主观评价，然后降主观评价的得分进行平均，就得到了 Mean Opinion Score（MOS）。

### Learning-based Perceptual Quality

为了消除主观的映像不一致的干扰，人们试图学习一个能评价图片质量的模型。例如 DeepQA 学习一个三元组：（失真图片，主观的错误得分映射，主观得分）。LPIPS 通过比较深度学习的提取的特征和图像感知区块的区别进行评价，结论是使用 CNNs 的模型比没有使用 CNNs 的模型提取的特征感知要好。

虽然这些方法展示了更好的获取人类视觉感知的方法，但是我们仍然不知道我们需要哪种质量的感知。主观的 IQA 评价仍然是主流。

### Task-based Evaluation

利用其它的任务进行评价重构的图像也是一种方法，有研究者同时将原始图像和重构的图像输入模型，使用重构图像对预测结果的性能影响评价重构图像的质量。

## Operating Channels

SR 问题不仅会使用 RGB 色彩空间，同时 YCbCr 色彩空间也是常用的。在 YCbCr 空间中，Y、Cb、Cr 分别表示亮度、蓝色色差（blue-difference）、红色色差（red-differnece）。现在的色彩空间越来越倾向于使用 RGB，这里提到 YCbCr 是为了提醒，不同的色彩空间会导致评价结果的差异很大。